<a href="https://colab.research.google.com/github/angomoson/BostonHousePricing/blob/main/mt5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers datasets
!pip install evaluate
!pip install rouge_score

In [ ]:
import os
import pandas as pd
import torch
from datasets import Dataset
import evaluate  # ✅ Use `evaluate` instead of `datasets.load_metric`
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq,
    TrainingArguments, Trainer
)

# ✅ Ensure CUDA is available if using GPU
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# ✅ Load the dataset
dataset_path = "your_dataset.csv"  # Change this to your actual dataset path
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Dataset file '{dataset_path}' not found!")

print("Loading dataset...")
df = pd.read_csv(dataset_path, encoding="utf-8").dropna().reset_index(drop=True)

# ✅ Ensure the dataset has the required columns
required_columns = {"article", "summary"}
if not required_columns.issubset(df.columns):
    raise ValueError(f"Dataset must have columns: {required_columns}. Found: {df.columns}")

# ✅ Convert dataset to Hugging Face format
dataset = Dataset.from_pandas(df)

# ✅ Load the tokenizer for mT5
model_name = "google/mt5-small"  # You can also try mt5-base or mt5-large for better performance
tokenizer = AutoTokenizer.from_pretrained(model_name)

# ✅ Tokenization function
def tokenize_function(examples, max_length=128):
    """Tokenizes input articles and summaries for mT5"""
    inputs = tokenizer(
        examples["article"], max_length=max_length, padding="max_length", truncation=True
    )
    targets = tokenizer(
        examples["summary"], max_length=max_length, padding="max_length", truncation=True
    )

    inputs["labels"] = targets["input_ids"]  # Target labels for summarization
    return inputs

# ✅ Apply tokenization
print("Tokenizing dataset...")
tokenized_dataset = dataset.map(tokenize_function, batched=True)

# ✅ Split dataset into training & validation sets
split_ratio = 0.9
train_size = int(len(tokenized_dataset) * split_ratio)

train_dataset = tokenized_dataset.select(range(train_size))
eval_dataset = tokenized_dataset.select(range(train_size, len(tokenized_dataset)))

print(f"Dataset split: {len(train_dataset)} training samples, {len(eval_dataset)} validation samples.")

# ✅ Load the mT5 Model for Summarization
print("Loading model...")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

# ✅ Use Data Collator for Padding
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# ✅ Define Training Arguments
training_args = TrainingArguments(
    output_dir="./mt5_summarization",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    learning_rate=3e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    report_to="none",
    fp16=torch.cuda.is_available(),
)

# ✅ Fix: Use `evaluate` Instead of `load_metric`
rouge = evaluate.load("rouge")

def compute_metrics(eval_pred):
    """Computes ROUGE scores for evaluation"""
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return {k: round(v.mid.fmeasure * 100, 2) for k, v in result.items()}

# ✅ Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

# ✅ Start Training
print("\n🚀 Starting training...")
trainer.train()

# ✅ Save the Fine-Tuned Model
print("\n✅ Training complete! Saving model...")
model.save_pretrained("mt5_finetuned")
tokenizer.save_pretrained("mt5_finetuned")

# ✅ Test the Fine-Tuned Model
print("\n🔹 Testing model with a sample article...")
sample_text = "Your sample article text here"  # Replace with actual text
inputs = tokenizer(sample_text, return_tensors="pt", truncation=True, padding="max_length", max_length=128).to(device)

# Generate summary
model.eval()
with torch.no_grad():
    output = model.generate(**inputs, max_length=50)

summary = tokenizer.decode(output[0], skip_special_tokens=True)
print("\n📝 Generated Summary:", summary)
